# In-depth Analysis of Successful Negotiation Cases

View complete details of successful negotiations, including:
- Complete conversation transcript
- Deal Terms reached
- Memory (M) state
- Plan (P) strategy
- Negotiation process analysis

## 1. Setup and Imports

In [53]:
import sqlite3
import pandas as pd
import json
from datetime import datetime
from IPython.display import display, HTML, Markdown
import warnings
warnings.filterwarnings('ignore')

print("✅ Import successful")

✅ Import successful


## 2. Connect to Database

In [76]:
# Modify the path here
DB_PATH = "negotiations_11_30.db"  # Change to your database path

conn = sqlite3.connect(DB_PATH)
print(f"✅ Connected to database: {DB_PATH}")

✅ Connected to database: negotiations_11_30.db


In [78]:
# Check what time columns exist
query = "PRAGMA table_info(negotiation_sessions)"
columns_info = pd.read_sql_query(query, conn)
print("negotiation_sessions table columns:")
display(columns_info[['name', 'type']])

negotiation_sessions table columns:


,name,type
0,session_id,TEXT
1,student_id,TEXT
2,student_name,TEXT
3,scenario_name,TEXT
4,student_role,TEXT
5,ai_role,TEXT
6,ai_model,TEXT
7,student_goes_first,BOOLEAN
8,use_memory,BOOLEAN
9,use_plan,BOOLEAN


In [79]:
# Check time format
query = """
SELECT 
    session_id,
    student_name,
    created_at,
    updated_at,
    total_rounds,
    deal_reached
FROM negotiation_sessions
LIMIT 5
"""

sample_times = pd.read_sql_query(query, conn)
display(sample_times)

,session_id,student_name,created_at,updated_at,total_rounds,deal_reached
0,1144aa88-511c-4b76-a560-a216efccb98c,Chris,2025-11-15T07:14:40.297549,2025-11-15T07:45:46.254733,6,0
1,65194c01-2d5b-4e6e-bc84-68cd534978c6,Chris,2025-11-15T08:55:59.726561,2025-11-15T08:56:57.576737,6,0
2,33b750dd-ea59-42ee-bba1-33035a4ba6b0,Chris,2025-11-15T08:56:20.962549,2025-11-15T08:57:11.924228,6,0
3,0616b831-318d-40e4-9890-7fded6cc67ab,Chris,2025-11-15T07:51:27.472206,2025-11-15T08:57:31.439587,6,1
4,de114cde-f8eb-4a78-8726-4a3daf07fa9c,Chris,2025-11-15T09:00:20.503126,2025-11-15T09:00:20.503296,6,0


In [80]:
# Get all negotiation records with duration
query = """
SELECT 
    session_id,
    student_name,
    scenario_name,
    student_role,
    ai_model,
    use_memory,
    use_plan,
    deal_reached,
    total_rounds,
    created_at,
    updated_at
FROM negotiation_sessions
ORDER BY created_at DESC
"""

all_df = pd.read_sql_query(query, conn)

# Convert to datetime and calculate duration
all_df['created_at'] = pd.to_datetime(all_df['created_at'])
all_df['updated_at'] = pd.to_datetime(all_df['updated_at'])
all_df['duration_seconds'] = (all_df['updated_at'] - all_df['created_at']).dt.total_seconds()
all_df['duration_minutes'] = all_df['duration_seconds'] / 60

print(f"\n📊 Total: {len(all_df)} negotiation records")
print(f"✅ Successful: {len(all_df[all_df['deal_reached']==1])} records")
print(f"❌ Failed: {len(all_df[all_df['deal_reached']==0])} records\n")

# Display list
display_df = all_df.copy()
display_df['session_id_short'] = display_df['session_id'].str[:8]
display_df['M+P'] = display_df.apply(
    lambda x: '✅' if x['use_memory'] and x['use_plan'] else '❌', 
    axis=1
)
display_df['deal'] = display_df['deal_reached'].apply(
    lambda x: '✅' if x == 1 else '❌'
)
display_df['duration_min'] = display_df['duration_minutes'].round(1)

display(display_df[['session_id_short', 'student_name', 'scenario_name', 
                     'ai_model', 'M+P', 'deal', 'total_rounds', 
                     'duration_min', 'created_at']])

print("\n💡 Copy session_id_short to view details")
print(f"\n⏱️ Average duration: {all_df['duration_minutes'].mean():.1f} minutes")
print(f"⏱️ Median duration: {all_df['duration_minutes'].median():.1f} minutes")


📊 Total: 84 negotiation records
✅ Successful: 15 records
❌ Failed: 69 records



,session_id_short,student_name,scenario_name,ai_model,M+P,deal,total_rounds,duration_min,created_at
0,31b608ca,Chris,Main_Street,openai/gpt-5,❌,❌,6,0.0,2025-11-30 03:56:35.707319
1,e71b6d79,Chris,Top_talent,openai/gpt-5,❌,❌,6,0.0,2025-11-30 03:52:58.627590
2,7b3a0797,Chris,Main_Street,openai/gpt-5,❌,❌,6,0.4,2025-11-30 03:33:25.577421
3,4df67a14,Chris,Main_Street,openai/gpt-5,❌,❌,6,0.7,2025-11-30 03:23:06.979750
4,b5244230,Chris,Main_Street,openai/gpt-5,❌,❌,6,0.2,2025-11-30 02:51:52.735390
5,bc29866a,Chris,Main_Street,openai/gpt-5,❌,❌,6,0.0,2025-11-30 02:42:05.189919
6,c9df6ffd,Catherine Xu,Top_talent,openai/gpt-5,❌,❌,6,25.2,2025-11-26 23:20:50.943579
7,9fcefd67,Catherine Xu,Top_talent,x-ai/grok-3,❌,✅,6,20.4,2025-11-26 17:20:01.114715
8,d2006a2e,Francisco Fernandez,Top_talent,openai/gpt-5,❌,✅,10,250.0,2025-11-26 00:21:05.219473
9,c7a23c0c,Francisco Fernandez,Top_talent,openai/gpt-5,❌,❌,6,0.0,2025-11-25 19:38:01.721284



💡 Copy session_id_short to view details

⏱️ Average duration: 12.7 minutes
⏱️ Median duration: 1.8 minutes


## 3. List All Successful Negotiations

In [75]:
# Get all successful negotiations
query = """
SELECT 
    session_id,
    student_name,
    scenario_name,
    student_role,
    ai_model,
    use_memory,
    use_plan,
    created_at,
    feedback_text TEXT
FROM negotiation_sessions
WHERE deal_reached = 1
ORDER BY created_at DESC
"""

successful_df = pd.read_sql_query(query, conn)

print(f"\n📊 Found {len(successful_df)} successful negotiations\n")

# Display list
display_df = successful_df.copy()
display_df['session_id_short'] = display_df['session_id'].str[:8]
display_df['M+P'] = display_df.apply(
    lambda x: '✅' if x['use_memory'] and x['use_plan'] else '❌', 
    axis=1
)

display(display_df[['session_id_short', 'student_name', 'scenario_name', 
                     'ai_model', 'M+P', 'created_at']])

print("\n💡 Copy session_id_short to view details")


📊 Found 11 successful negotiations



,session_id_short,student_name,scenario_name,ai_model,M+P,created_at
0,6bcbd0fe,Chris,Main_Street,openai/gpt-5,❌,2025-12-01T06:59:07.400156
1,03b5791c,Chris,Main_Street,openai/gpt-5,✅,2025-12-01T06:54:53.616963
2,06eb1797,Chris,Top_talent,openai/gpt-5,❌,2025-12-01T06:54:01.765574
3,d9e5ab85,Chris,Top_talent,openai/gpt-5,❌,2025-12-01T06:53:49.801746
4,2d6e0616,Chris,Main_Street,openai/gpt-5,❌,2025-12-01T06:17:38.355369
5,30c3c80c,Chris,Main_Street,openai/gpt-5,❌,2025-12-01T06:17:23.112858
6,746dbd3e,Chris,Top_talent,openai/gpt-5,✅,2025-11-30T23:21:57.375665
7,05a9b9d7,Chris,Top_talent,openai/gpt-5,✅,2025-11-30T23:21:51.933084
8,c5ab236a,Chris,Top_talent,openai/gpt-5,✅,2025-11-30T23:21:49.456787
9,21d35936,Chris,Top_talent,openai/gpt-5,✅,2025-11-30T23:21:41.849373



💡 Copy session_id_short to view details


In [71]:
query = """
SELECT 
    student_name,
    COUNT(*) as count
FROM negotiation_sessions
GROUP BY student_name
ORDER BY count DESC
"""

name_counts = pd.read_sql_query(query, conn)
print(f"📊 总共有 {len(name_counts)} 个不同的学生\n")
display(name_counts)

📊 总共有 8 个不同的学生



,student_name,count
0,Chris,72
1,Francisco Fernandez,3
2,Zihao Sun,2
3,Catherine Xu,2
4,AL,2
5,sasha,1
6,Runzhi Jin,1
7,Prof Cain,1


## 4. View Specific Case Details

### 👇 Enter the session_id you want to view below (first 8 characters are sufficient)

In [64]:
# ============================================
# 👉 MODIFY HERE: Enter the session_id you want to view
# ============================================
SESSION_ID = "db6d85c2-5ccb-4df7-a62c-89cf210bd1cb"  # Change to the actual session_id (first 8 characters)

# If only first 8 characters, find the complete ID
if len(SESSION_ID) == 8:
    full_id_query = f"""
    SELECT session_id FROM negotiation_sessions 
    WHERE session_id LIKE '{SESSION_ID}%' AND deal_reached = 1
    """
    result = pd.read_sql_query(full_id_query, conn)
    if len(result) > 0:
        SESSION_ID = result['session_id'].iloc[0]
        print(f"✅ Found complete ID: {SESSION_ID}")
    else:
        print(f"❌ No successful cases found starting with {SESSION_ID} ")

# Get complete data
query = f"""
SELECT * FROM negotiation_sessions
WHERE session_id = '{SESSION_ID}'
"""

case_df = pd.read_sql_query(query, conn)

if len(case_df) == 0:
    print(f"❌ session_id not found: {SESSION_ID}")
else:
    case = case_df.iloc[0]
    print(f"\n✅ Case loaded: {SESSION_ID}")


✅ Case loaded: db6d85c2-5ccb-4df7-a62c-89cf210bd1cb


## 5. 📋 Case Overview

In [22]:
if len(case_df) > 0:
    print("=" * 80)
    print("                          Case Overview")
    print("=" * 80)
    
    print(f"\n🆔 Session ID:     {case['session_id']}")
    print(f"👤 Student:           {case['student_name']} (ID: {case['student_id']})")
    print(f"🎯 Scenario:           {case['scenario_name']}")
    print(f"🎭 Student Role:       {case['student_role']}")
    print(f"🤖 AI Role:         {case['ai_role']}")
    print(f"🧠 AI Model:         {case['ai_model']}")
    print(f"🔧 Memory Enabled:     {'Yes ✅' if case['use_memory'] else 'No ❌'}")
    print(f"📋 Plan Enabled:       {'Yes ✅' if case['use_plan'] else 'No ❌'}")
    print(f"🎲 Who Starts First:       {'student' if case['student_goes_first'] else 'AI'}")
    print(f"🔄 Total Rounds:       {case['total_rounds']}")
    
    # 计算时长
    try:
        start = pd.to_datetime(case['created_at'])
        end = pd.to_datetime(case['updated_at'])
        duration = (end - start).total_seconds() / 60
        print(f"⏳ Duration:           {duration:.1f} minutes")
    except:
        pass
    
    # 消息统计
    try:
        transcript = json.loads(case['transcript'])
        print(f"💬 Total Messages:       {len(transcript)}")
        print(f"📅 Start Time:       {case['created_at']}")
        print(f"✅ Completion Time:       {case['updated_at']}")
    except:
        pass

                          Case Overview

🆔 Session ID:     03b5791c-d8c7-480f-9810-db0f6653bd45
👤 Student:           Chris (ID: 123)
🎯 Scenario:           Main_Street
🎭 Student Role:       side2
🤖 AI Role:         side1
🧠 AI Model:         openai/gpt-5
🔧 Memory Enabled:     Yes ✅
📋 Plan Enabled:       Yes ✅
🎲 Who Starts First:       student
🔄 Total Rounds:       6
⏳ Duration:           36.3 minutes
💬 Total Messages:       13
📅 Start Time:       2025-12-01T06:54:53.616963
✅ Completion Time:       2025-12-01T07:31:09.857781


## 6. 🤝 Deal Terms Reached

In [65]:
if len(case_df) > 0:
    print("=" * 80)
    print("                          Deal Terms")
    print("=" * 80)
    
    print("\n📝 Student's Proposed Deal Terms:")
    print("-" * 80)
    if case['student_deal_json']:
        try:
            student_deal = json.loads(case['student_deal_json'])
            print(json.dumps(student_deal, indent=2, ensure_ascii=False))
        except Exception as e:
            print(f"Parse error: {e}")
    else:
        print("(Not recorded)")
    
    print("\n\n🤖 AI Confirmed Deal Terms:")
    print("-" * 80)
    if case['ai_deal_json']:
        try:
            ai_deal = json.loads(case['ai_deal_json'])
            print(json.dumps(ai_deal, indent=2, ensure_ascii=False))
        except Exception as e:
            print(f"Parse error: {e}")
    else:
        print("(Not recorded)")
    
    # 对比差异
    if case['student_deal_json'] and case['ai_deal_json']:
        try:
            student_deal = json.loads(case['student_deal_json'])
            ai_deal = json.loads(case['ai_deal_json'])
            
            print("\n\n🔍 Comparison Analysis:")
            print("-" * 80)
            
            all_keys = set(student_deal.keys()) | set(ai_deal.keys())
            
            for key in sorted(all_keys):
                student_val = student_deal.get(key, "(Missing)")
                ai_val = ai_deal.get(key, "(Missing)")
                
                if student_val == ai_val:
                    print(f"✅ {key:30s}: {student_val}")
                else:
                    print(f"⚠️  {key:30s}:")
                    print(f"   Student: {student_val}")
                    print(f"   AI:   {ai_val}")
        except:
            pass

                          Deal Terms

📝 Student's Proposed Deal Terms:
--------------------------------------------------------------------------------
(Not recorded)


🤖 AI Confirmed Deal Terms:
--------------------------------------------------------------------------------
(Not recorded)


In [60]:
case["ai_memory_history"]

'[]'

In [59]:
case["ai_plan_history"]

'[{"round": "1.1", "content": "- ROUND GOAL: Set a firm low anchor at $420,000; surface Rosalind\\u2019s urgency and target a sub-$550,000 negotiation lane.\\n- KEY LEVERS: Soft local market; conversion/CapEx to office use; rental comps; vacancy/tenant risk; interest rates; certainty/fast close; her eagerness/timing.\\n- TACTICS: Open warm but businesslike; present $420k immediately with brief rationale (comps ~475\\u2013525, CapEx 100\\u2013150k, rates, niche past use); emphasize cash and 14-day close; do not reveal adjacency/unique value; ask 3 tight probes (why selling/timing, other offers, must-have number); resist moving off anchor this round and invite her counter.\\n- OFFER SCAFFOLD: $420,000 purchase price, as-is, cash, 14-day close, minimal paperwork, offer good for 48 hours."}]'

## 7. 🧠 Memory (M) State

In [61]:
if len(case_df) > 0:
    print("=" * 80)
    print("                     AI Memory (Final State)")
    print("=" * 80)
    
    if not case['use_memory']:
        print("\n❌ Memory not enabled for this case")
    elif not case['ai_memory']:
        print("\n(Memory state not recorded)")
    else:
        print("\n🧠 AI Memory recorded the following information:\n")
        print(case['ai_memory'])
        
        # 分析Memory内容
        memory_text = case['ai_memory']
        print("\n\n📊 Memory Analysis:")
        print("-" * 80)
        print(f"Character count: {len(memory_text)}")
        print(f"Line count: {len(memory_text.split('\n'))}")
        
        # 检查关键词
        keywords = ['OFFERS', 'PATTERNS', 'PRIORITIES', 'CONSTRAINTS']
        found = [kw for kw in keywords if kw in memory_text]
        print(f"Key sections included: {', '.join(found)}")

                     AI Memory (Final State)

(Memory state not recorded)


## 8. 📋 Plan (P) Strategy

In [62]:
if len(case_df) > 0:
    print("=" * 80)
    print("                   AI Strategic Plan (Final State)")
    print("=" * 80)
    
    if not case['use_plan']:
        print("\n❌ Plan not enabled for this case")
    elif not case['ai_plan']:
        print("\n(Plan state not recorded)")
    else:
        print("\n📋 AI's Final Strategic Plan:\n")
        print(case['ai_plan'])
        
        # 分析Plan内容
        plan_text = case['ai_plan']
        print("\n\n📊 Plan Analysis:")
        print("-" * 80)
        print(f"Character count: {len(plan_text)}")
        print(f"Line count: {len(plan_text.split('\n'))}")
        
        # 检查关键词
        keywords = ['ROUND GOAL', 'KEY LEVERS', 'TACTICS', 'OFFER SCAFFOLD', 'RISK']
        found = [kw for kw in keywords if kw in plan_text]
        print(f"Strategy sections included: {', '.join(found)}")

                   AI Strategic Plan (Final State)

📋 AI's Final Strategic Plan:

- ROUND GOAL: Set a firm low anchor at $420,000; surface Rosalind’s urgency and target a sub-$550,000 negotiation lane.
- KEY LEVERS: Soft local market; conversion/CapEx to office use; rental comps; vacancy/tenant risk; interest rates; certainty/fast close; her eagerness/timing.
- TACTICS: Open warm but businesslike; present $420k immediately with brief rationale (comps ~475–525, CapEx 100–150k, rates, niche past use); emphasize cash and 14-day close; do not reveal adjacency/unique value; ask 3 tight probes (why selling/timing, other offers, must-have number); resist moving off anchor this round and invite her counter.
- OFFER SCAFFOLD: $420,000 purchase price, as-is, cash, 14-day close, minimal paperwork, offer good for 48 hours.


📊 Plan Analysis:
--------------------------------------------------------------------------------
Character count: 740
Line count: 4
Strategy sections included: ROUND GOAL, KE

## 9. 💬 Complete Conversation Transcript（Transcript）

In [63]:
if len(case_df) > 0:
    print("=" * 80)
    print("                         Complete Conversation Transcript")
    print("=" * 80)
    
    try:
        transcript = json.loads(case['transcript'])
        total_messages = len(transcript)
        
        print(f"\n💬 共 {total_messages} 条消息\n")
        print("=" * 80)
        
        for i, message in enumerate(transcript, 1):
            # 解析消息格式: "Round X.Y - Label: content"
            if " - " in message:
                parts = message.split(" - ", 1)
                round_info = parts[0]
                
                if len(parts) > 1 and ":" in parts[1]:
                    speaker_content = parts[1].split(":", 1)
                    speaker = speaker_content[0].strip()
                    content = speaker_content[1].strip() if len(speaker_content) > 1 else ""
                    
                    # 判断是学生还是AI
                    if case['student_role'] in speaker or "Student" in speaker:
                        icon = "👤"
                        print(f"\n{icon} [{round_info}] {speaker}:")
                    else:
                        icon = "🤖"
                        print(f"\n{icon} [{round_info}] {speaker}:")
                    
                    # 打印内容（处理长消息）
                    if len(content) > 500:
                        print(f"   {content[:2000]}...")
                        print(f"   ... (共 {len(content)} characters)")
                    else:
                        print(f"   {content}")
                else:
                    print(f"\n💬 [{round_info}]")
                    print(f"   {parts[1] if len(parts) > 1 else message}")
            else:
                print(f"\n💬 消息 {i}:")
                if len(message) > 500:
                    print(f"   {message[:500]}...")
                    print(f"   ... (共 {len(message)} characters)")
                else:
                    print(f"   {message}")
            
            print("-" * 80)
        
    except Exception as e:
        print(f"\n❌ 解析transcript出错: {e}")

                         Complete Conversation Transcript

💬 共 1 条消息


🤖 [Round 1.1] Fred Starr (Buyer):
   Rosalind—good to see you. I’ve always known you to run a tight ship, so I’ll get straight to brass tacks.

I’m prepared to buy 123 Main for $420,000, as-is. Cash, 14-day close, minimal fuss. The town’s triplex comps I’m seeing are in the 475–525 range depending on condition, but rates are still biting, and converting a former specialty retail/resi setup into proper office-compliant space means real money—fire separations, accessibility, parking—easily six figures. Between that and vacancy risk during a reposition, $420k is the number that pencils for me. Offer stands 48 hours.

A couple quick questions so I can understand where you are:
- What’s got you selling now—anything time-sensitive on your end?
- Are there other offers on the table, or a number you feel you must hit?
- How quickly are you looking to wrap this up?

If the certainty and speed matter to you, I can move immedi

In [106]:
case['feedback_text']

'═══════════════════════════════════════\n📊 PERFORMANCE SUMMARY\n═══════════════════════════════════════\n\n- Final outcome: 31,000 points achieved\n- Overall assessment: You secured a deal above your BATNA of 0 points, which is a positive result under time constraints. However, you left significant value on the table by conceding on high-point issues early and not pushing harder on key financial terms.\n\n═══════════════════════════════════════\n🎯 STRATEGIC ANALYSIS\n═══════════════════════════════════════\n\n- **Start Date (15,000 points achieved out of 15,000 possible)**: You secured the maximum value by holding firm on August 1st or later. Your consistent emphasis on personal timing needs paid off as the recruiter conceded early. This was a strong tactical move, ensuring a high-value win without trade-offs.\n  \n- **Work Location (0 points achieved out of 4,000 possible)**: You conceded to New Haven (0 points) despite Philadelphia offering 4,000 points. While you articulated person

## 10. 📊 Negotiation Process Analysis

In [58]:
if len(case_df) > 0:
    print("=" * 80)
    print("                        Negotiation Process Analysis")
    print("=" * 80)
    
    try:
        transcript = json.loads(case['transcript'])
        total_messages = len(transcript)
        
        # 统计消息长度
        message_lengths = []
        student_messages = 0
        ai_messages = 0
        deal_signals = 0
        
        for msg in transcript:
            # 计算消息长度
            if " - " in msg:
                parts = msg.split(" - ", 1)
                if len(parts) > 1:
                    msg_content = parts[1]
                    message_lengths.append(len(msg_content))
            else:
                message_lengths.append(len(msg))
            
            # 统计发言者
            if case['student_role'] in msg:
                student_messages += 1
            elif case['ai_role'] in msg:
                ai_messages += 1
            
            # 检查deal信号
            if "$DEAL_REACHED$" in msg:
                deal_signals += 1
        
        print("\n📊 Basic Statistics:")
        print("-" * 80)
        print(f"Total Messages:        {total_messages}")
        print(f"Student Messages:        {student_messages}")
        print(f"AI Messages:          {ai_messages}")
        print(f"完成Round数:      {total_messages // 2}")
        print(f"$DEAL_REACHED$:  出现 {deal_signals} 次")
        
        if message_lengths:
            print("\n📏 Message Complexity:")
            print("-" * 80)
            print(f"Average Message Length:    {sum(message_lengths)/len(message_lengths):.0f} characters")
            print(f"Shortest Message:        {min(message_lengths)} characters")
            print(f"Longest Message:        {max(message_lengths)} characters")
        
        # 计算每Round平均时长
        try:
            start = pd.to_datetime(case['created_at'])
            end = pd.to_datetime(case['updated_at'])
            total_minutes = (end - start).total_seconds() / 60
            rounds = total_messages // 2
            
            if rounds > 0:
                print("\n⏱️  Time Analysis:")
                print("-" * 80)
                print(f"Total Duration:          {total_minutes:.1f} minutes")
                print(f"每Round平均:      {total_minutes/rounds:.1f} minutes")
                print(f"Average Per Message:    {total_minutes/total_messages:.1f} minutes")
        except:
            pass
        
        # 判断谈判风格
        print("\n🎯 Negotiation Characteristics:")
        print("-" * 80)
        
        avg_length = sum(message_lengths)/len(message_lengths) if message_lengths else 0
        if avg_length > 500:
            print("💬 Message Style:      Detailed (longer messages)")
        elif avg_length > 200:
            print("💬 Message Style:      Moderate")
        else:
            print("💬 Message Style:      Concise (shorter messages)")
        
        if total_minutes / rounds < 3 if rounds > 0 else False:
            print("⚡ Negotiation Pace:      快速（<3minutes/Round）")
        elif total_minutes / rounds < 5 if rounds > 0 else False:
            print("⚡ Negotiation Pace:      Normal（3-5minutes/Round）")
        else:
            print("⚡ Negotiation Pace:      缓慢（>5minutes/Round）")
        
        if rounds < case['total_rounds'] / 2:
            print("🎉 Deal Efficiency:      高（不到一半Round就达成）")
        elif rounds < case['total_rounds']:
            print("🎉 Deal Efficiency:      Normal")
        else:
            print("🎉 Deal Efficiency:      压线（用完所有Round）")
        
    except Exception as e:
        print(f"\n❌ Analysis error: {e}")

## 11. 🔍 Quick View: Comparison of Recent 3 Successful Cases

In [19]:
# 获取最近3个成功案例
recent_query = """
SELECT 
    session_id,
    student_name,
    scenario_name,
    ai_model,
    use_memory,
    use_plan,
    transcript,
    created_at,
    updated_at
FROM negotiation_sessions
WHERE deal_reached = 1
ORDER BY created_at DESC
LIMIT 3
"""

recent_df = pd.read_sql_query(recent_query, conn)

if len(recent_df) > 0:
    print("=" * 80)
    print("                 Comparison of Recent 3 Successful Cases")
    print("=" * 80)
    
    comparison_data = []
    
    for idx, row in recent_df.iterrows():
        try:
            transcript = json.loads(row['transcript'])
            msg_count = len(transcript)
            
            start = pd.to_datetime(row['created_at'])
            end = pd.to_datetime(row['updated_at'])
            duration = (end - start).total_seconds() / 60
            
            comparison_data.append({
                'Session': row['session_id'][:8],
                'Student': row['student_name'],
                'Scenario': row['scenario_name'],
                'Model': row['ai_model'].split('/')[-1],
                'M+P': '✅' if row['use_memory'] and row['use_plan'] else '❌',
                'Messages': msg_count,
                'Duration(min)': f"{duration:.1f}",
                'Rounds': msg_count // 2
            })
        except:
            pass
    
    if comparison_data:
        comparison_df = pd.DataFrame(comparison_data)
        display(comparison_df)
else:
    print("\n❌ No successful cases found")

                 Comparison of Recent 3 Successful Cases


,Session,Student,Scenario,Model,M+P,Messages,Duration(min),Rounds
0,6bcbd0fe,Chris,Main_Street,gpt-5,❌,10,26.1,5
1,03b5791c,Chris,Main_Street,gpt-5,✅,13,36.3,6
2,06eb1797,Chris,Top_talent,gpt-5,❌,8,23.0,4


## 12. Close Database Connection

In [13]:
conn.close()
print("✅ Database connection closed")

✅ Database connection closed
